<a href="https://colab.research.google.com/github/DurdenTyler2008/University-of-AI/blob/main/16_basketball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 

from tensorflow.keras.models import Sequential, Model  
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten 
from tensorflow.keras import utils #Используем для to_categoricall
from tensorflow.keras.optimizers import Adam,Adadelta,SGD,Adagrad,RMSprop 
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.callbacks import LambdaCallback # подключаем колбэки

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split # Для разбивки на выборки
from sklearn.metrics import mean_squared_error, mean_absolute_error #
       
import random            #Для генерации случайных чисел 
import math              # Для округления
import os                #Для работы с файлами 
import re 
import matplotlib.pyplot as plt 
%matplotlib inline

from IPython.display import clear_output


In [2]:
from google.colab import drive # Подключаем диск
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive"    # читаю директории

 1_2.jpg	       cars_new.csv	      sonar.csv
 14text.txt	       city_temperature.csv   train.csv
 2_2.jpg	      'Colab Notebooks'       train_vector_1.csv
 3_2.jpg	       Diseases		      train_vector_1.gsheet
 3_3.jpg	       file_csv.csv	      unknown
 4_2.jpg	       hh_fixed.csv	      wine_base.csv
 4.jpg		       iris.csv		      zakupki_hack.csv
 7_2_kolorpaint.jpeg   lkk_meters.csv	      Автомобили.zip
 7_kolorpaint.jpeg    'Matplotlib&Seaborn'    Болезни.zip
 7kolorpaint.png       moscow.csv	     'Задача 2_6.png'
 AIFramework	       Pandas1_ultra-pro     'Тексты писателей.zip'
 basketball.csv        Pandas2


**Предсказание результата баскетбольного матча, по промежуточному результату**



In [4]:
df = pd.read_csv('/content/drive/My Drive/basketball.csv', encoding= 'cp1251', sep=';', header=0, index_col=0) 
print (df.shape)
df.head(10) 

(52450, 9)


,TOTAL,info,Ком. 1,Ком. 2,Минута,Общая минута,Секунда,fcount,ftime
0,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,0.0,1,1.0,30,81,90.0
1,"100,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,1,1.0,45,81,105.0
2,"99,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,0,81,120.0
3,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,30,81,150.0
4,"95,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,3,3.0,0,81,180.0
5,"91,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,4,4.0,0,81,240.0
6,"93,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,4,4.0,4,4.0,30,81,270.0
7,"90,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,4,4.0,6,6.0,0,81,360.0
8,"91,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,5,4.0,6,6.0,0,81,360.0
9,"92,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,5,5.0,6,6.0,0,81,360.0


Total - предсказание алгоритма владельцев базы. Info - данные о матче. Ком.1 и Ком.2 - забитые мячи. Минута и Секунда - прошедшее время тайма. Ftime - прошедшее время игры в секундах без разбивки на таймы.

Fcount - финальный результат игры - надо предсказать. 

In [5]:
Xtrain = np.array(df[['Ком. 1','Ком. 2', 'ftime']].astype('int'))    # Преобразуем данные в numpy
Ytrain = np.array(df['fcount'].astype('int'))                        # Преобразуем данные в numpy

In [6]:
x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size=0.05) # Разбиваем по выборкам

In [10]:
model = Sequential() 

model.add(Dense(200,input_dim=3, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="MAE", optimizer="adam", metrics=["mae"]) 
# можно использовать MAE (средняя абсолютная ошибка) или MSE(средняя квадратичная ошибка), результат примерно одинаковый

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               800       
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 41,201
Trainable params: 41,201
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(x_train, y_train, batch_size=500, epochs=40,validation_data=(x_test, y_test),verbose=1, shuffle=True)

Epoch 1/40
100/100 [==============================] - 1s 4ms/step - loss: 79.1516 - mae: 79.1516 - val_loss: 60.3097 - val_mae: 60.3097
Epoch 2/40
100/100 [==============================] - 0s 3ms/step - loss: 60.0611 - mae: 60.0611 - val_loss: 58.6969 - val_mae: 58.6969
Epoch 3/40
100/100 [==============================] - 0s 3ms/step - loss: 58.6332 - mae: 58.6332 - val_loss: 57.9748 - val_mae: 57.9748
Epoch 4/40
100/100 [==============================] - 0s 3ms/step - loss: 57.1845 - mae: 57.1845 - val_loss: 57.4200 - val_mae: 57.4200
Epoch 5/40
100/100 [==============================] - 0s 3ms/step - loss: 56.3242 - mae: 56.3242 - val_loss: 54.3630 - val_mae: 54.3630
Epoch 6/40
100/100 [==============================] - 0s 3ms/step - loss: 53.8314 - mae: 53.8314 - val_loss: 55.5890 - val_mae: 55.5890
Epoch 7/40
100/100 [==============================] - 0s 3ms/step - loss: 51.6308 - mae: 51.6308 - val_loss: 47.4270 - val_mae: 47.4270
Epoch 8/40
100/100 [============================

In [14]:
pred = model.predict(x_test)
for i in range(len(pred[:10])): # Выводим результаты
    print('Данные -',x_test[i],'  Предсказание - ',pred[i],'   Результат игры - ', y_test[i])

Данные - [ 12  14 705]   Предсказание -  [115.99476]    Результат игры -  81
Данные - [ 26  19 735]   Предсказание -  [141.27605]    Результат игры -  141
Данные - [  45   33 1500]   Предсказание -  [116.54158]    Результат игры -  133
Данные - [  50   61 1470]   Предсказание -  [161.15181]    Результат игры -  192
Данные - [  66   74 2145]   Предсказание -  [143.05092]    Результат игры -  150
Данные - [  2   2 120]   Предсказание -  [142.83456]    Результат игры -  153
Данные - [  59   60 2130]   Предсказание -  [121.40642]    Результат игры -  147
Данные - [  37   55 1725]   Предсказание -  [119.87292]    Результат игры -  119
Данные - [  6   0 120]   Предсказание -  [146.92491]    Результат игры -  155
Данные - [  40   29 1095]   Предсказание -  [137.99997]    Результат игры -  140
